So I've got all this data about a year ago for my dissertation, everything in SQL.
It's not exactly reproducible, but here is the code I've used for cleaning this data.

No need to run this file, as the result (`data.p`) is already checked in. 

In [14]:
import pandas as pd
import pickle

In [52]:
# Reading in data, basic cleaning 

df=pd.read_csv('../datafiles/stata.csv')

df = pd.get_dummies(df['CPO']).join(df)

del df['ID2']
del df['ReportedTotal']
del df['ReportedInd']
del df['Party']
del df['CPO']
df = df.fillna(0)
df.columns = df.columns.map(lambda x: x.lower())



In [68]:
df = pd.read_pickle('../datafiles/data.p')
a = pd.get_dummies(df['party']).rename(columns = {1: 'dem',
                                              2: 'rep'})
df = df.merge(a, left_index=True, right_index=True)
df = df.drop(['i', 'c', 'o', 'party', 'acceptpacs', 'seat' ], axis=1)

In [36]:
# Computing various statistics

col_list_unitem= ['unitemind', 'unitempacs', 'unitemfor', 'unitemproxy']
col_list_item = ['itemind', 'unitempacs', 'unitemfor', 'unitemproxy']
df['unitemtotal'] = df[col_list_unitem].sum(axis=1) 
df['itemtotal'] = df[col_list_item].sum(axis=1)
df['total'] = df[['unitemtotal', 'itemtotal']].sum(axis=1)
df['unitemie'] = df[['unitemfor', 'unitemproxy']].sum(axis=1)
df['itemie'] = df[['itemfor', 'itemproxy']].sum(axis=1)
df['percunitem'] = df['unitemtotal']/df['total']
df['total'] = df['total'].apply(lambda x: float(x))
df['percpacs']= df['totalpacs']/df['total']
df['ie'] = df['iefor'] + df['ieproxy']
df['percie'] = df['ie']/df['total']
df['percind']= df['totalind']/df['total']
df['percstate']= df['instate']/df['total']

In [ ]:
def rename_duplicate_columns(df):
    L = df.columns
    seen = {}
    result = []
    for column in L:
        if not column in seen:
            seen[column] = 1
            result.append(column)
        else:
            seen[column] +=1 
            result.append('%s_%d' % (column, seen[column]))
    df.columns = result
    return df

df = rename_duplicate_columns(df)
df = df.drop('congyear', axis =1)
df = df.rename(columns = {"congyear_2" : 'congyear'})


In [37]:
pickle.dump( df, open( "../datafiles/data.p", "wb" ) )